In [60]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from urllib import request


In [2]:
# Import Production Data (and State Index)
# Production Data Source
# https://www.kaggle.com/datasets/kevinmorgado/us-energy-generation-2001-2022?select=organised_Gen.csv

# Load the State Index CSV file
state_df = pd.read_csv("Resources/states.csv", usecols = ['State', 'Code'])

# Load the Production CSV file and exclude first column
col_list = ['YEAR', 'MONTH', 'STATE', 'TYPE OF PRODUCER', 'ENERGY SOURCE', 'GENERATION (Megawatthours)']
prod_df = pd.read_csv("Resources/organised_Gen.csv", usecols = col_list)

# Assign title case to column names and rename 'State'
col_list_cased = [col.title() for col in col_list]
prod_df.columns = col_list_cased

# Filter for years 2002 - 2021 to match investment data set
prod_df = prod_df.loc[(prod_df['Year'] > 2001) & (prod_df['Year'] < 2022) & (prod_df['Type Of Producer'] == "Total Electric Power Industry")]

# Filter for common energy types across both data sets
prod_source_list = ['Geothermal', 'Hydroelectric Conventional', 'Other Biomass', 'Solar Thermal and Photovoltaic', 'Wind']
prod_df = prod_df.loc[prod_df['Energy Source'].isin(prod_source_list)]

# Rename energy sources to common convention
prod_df = prod_df.replace(['Hydroelectric Conventional', 'Other Biomass', 'Solar Thermal and Photovoltaic'], 
                          ['Hydroelectric', 'Biomass', 'Solar'])

# Display sample data
prod_df.head(20)

,Year,Month,State,Type Of Producer,Energy Source,Generation (Megawatthours)
20215,2002,1,AK,Total Electric Power Industry,Hydroelectric,118944.0
20217,2002,1,AK,Total Electric Power Industry,Biomass,999.0
20242,2002,1,AL,Total Electric Power Industry,Hydroelectric,947959.0
20244,2002,1,AL,Total Electric Power Industry,Biomass,1622.0
20273,2002,1,AR,Total Electric Power Industry,Hydroelectric,230870.0
20275,2002,1,AR,Total Electric Power Industry,Biomass,288.0
20302,2002,1,AZ,Total Electric Power Industry,Hydroelectric,687079.0
20303,2002,1,AZ,Total Electric Power Industry,Solar,37.0
20304,2002,1,AZ,Total Electric Power Industry,Biomass,6063.0
20336,2002,1,CA,Total Electric Power Industry,Hydroelectric,2515525.0


In [6]:
# Import Investment Data
# Investment Data Source
# https://www.wctsservices.usda.gov/Energy/Downloads

# Load the Investment Excel file, "Detailed" Sheet
inv_df = pd.read_excel("Resources/EnergyInvestments_DataDownloads.xlsx", sheet_name = "Detailed")

# Filter for common energy types across both data sets
inv_source_list = ['Geothermal', 'Hydroelectric', 'Renewable Biomass', 'Solar', 'Wind']
inv_df = inv_df.loc[inv_df['Energy Type'].isin(inv_source_list)]

# Rename energy sources to common convention
inv_df = inv_df.replace('Renewable Biomass', 'Biomass')

# Join with State Index data to get State Abbreviation column
inv_df = pd.merge(inv_df, state_df, on = "State")

# Rename columns
inv_df.rename(columns={"State": "State Name", "Code": "State", "Energy Type": "Energy Source", "Program_Name": "Program Name"}, inplace = True)

# Display sample data
inv_df.head(20)

# Export to csv
inv_df.to_csv("output_data/inv.csv")

In [4]:
#### Exploratory Data Analysis

In [109]:
# Import Census Data
# Census Data Sources

# https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html
# https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html
# https://www.census.gov/programs-surveys/popest/technical-documentation/research/evaluation-estimates.2010.html#list-tab-Y660N3MTL49GQLLYDJ

years = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
        2017, 2018, 2019, 2020, 2021]

census_df = pd.DataFrame(columns=['State', 'Population', 'Year'])

state = []
population = []
iterationyear = []

for year in years:
        
    if year < 2011:
        
        popappend = "POPESTIMATE" + str(year)
        
        pop_df = pd.read_csv("Resources/co-est2010-alldata.csv", encoding='ISO-8859-1')
        pop_df = pop_df[[popappend, 'STNAME', 'CTYNAME']]
        pop_df = pop_df.rename(columns={popappend: "Population", "STNAME": "State"})
       
        # This specific dataset is broken down by city pop as well, but the full state pop
        # is held in a 'city' that is the state name, so dropping any rows where
        # the state name is not equal to the city name.
        pop_df = pop_df[pop_df.State == pop_df.CTYNAME]
       
        for ind in pop_df.index:
            state.append(pop_df['State'][ind])
            population.append(pop_df['Population'][ind])
            iterationyear.append(year)
            
    if (year > 2010 and year < 2020):
    
        popappend = "POPESTIMATE" + str(year)
        pop_df = pd.read_csv("Resources/nst-est2019-alldata.csv", encoding='ISO-8859-1')
        pop_df = pop_df[[popappend, 'NAME']]
        pop_df = pop_df.rename(columns={popappend: "Population", "NAME": "State"})
        
        for ind in pop_df.index:
            state.append(pop_df['State'][ind])
            population.append(pop_df['Population'][ind])
            iterationyear.append(year)
    
    if year > 2019:
    
        popappend = "POPESTIMATE" + str(year)
        pop_df = pd.read_csv("Resources/NST-EST2021-alldata.csv", encoding='ISO-8859-1')
        pop_df = pop_df[[popappend, 'NAME']]
        pop_df = pop_df.rename(columns={popappend: "Population", "NAME": "State"})
        
        for ind in pop_df.index:
            state.append(pop_df['State'][ind])
            population.append(pop_df['Population'][ind])
            iterationyear.append(year)
        
        
census_df['State'] = state
census_df['Population'] = population
census_df['Year'] = iterationyear
                                     
print(census_df)

cleaned_census_df = census_df.groupby(['State', 'Year']).sum()

print(cleaned_census_df)
cleaned_census_df.to_csv("output_data/census_data.csv")

              State  Population  Year
0           Alabama     4471462  2002
1            Alaska      641974  2002
2           Arizona     5451472  2002
3          Arkansas     2704471  2002
4        California    34867773  2002
...             ...         ...   ...
1090     Washington     7738692  2021
1091  West Virginia     1782959  2021
1092      Wisconsin     5895908  2021
1093        Wyoming      578803  2021
1094    Puerto Rico     3263584  2021

[1095 rows x 3 columns]
              Population
State   Year            
Alabama 2002     4471462
        2003     4489876
        2004     4510588
        2005     4542912
        2006     4594911
...                  ...
Wyoming 2017      578931
        2018      577601
        2019      578759
        2020      577267
        2021      578803

[1086 rows x 1 columns]
